In [1]:
using ITensors 
using Plots
using Measurements   
using DelimitedFiles 

In [2]:
# lattice_large - lattice_small = 2 
function GrossNeveu(L::Int64,N::Int64,m::Float64,G²::Float64)
    l = L-2;
    sites = siteinds("Fermion",L)
    ampoₗ = OpSum()
    ampoₛ = OpSum()
    # Large Lattice 
    for j in 1:L-N
        ampoₗ -= 1im,"C",j,"Cdag",j+N
        ampoₗ += 1im,"C",j+N,"Cdag",j
    end
    for j in 1:N:L-N
        ampoₗ += (G²/2),"C * Cdag",j
    end
    for j in 1:L
        ampoₗ += (m*(-1)^j),"C * Cdag",j
    end
    Hₗ = MPO(ampoₗ, sites)
    ψₗ₀ = randomMPS(sites,100)
    ######################################
    ## small lattice 
    for j in 1:l-N
        ampoₛ -= 1im,"C",j,"Cdag",j+N
        ampoₛ += 1im,"C",j+N,"Cdag",j
    end
    for j in 1:N:l-N
        ampoₛ += (G²/2),"C * Cdag",j
    end
    for j in 1:l
        ampoₛ += (m*(-1)^j),"C * Cdag",j 
    end
    Hₛ = MPO(ampoₛ,sites[1:l])
    ψₛ₀ = randomMPS(sites[1:l],100)
#     sweep = Sweeps(sw)
#     setmaxdim!(sweep,repeat([1024],sw)...)
#     setcutoff!(sweep,1E-10)
    sweep = Sweeps([
         "maxdim" "mindim" "cutoff" 
          100      10       1e-16    
          200      10      1e-16    
          500      20      1e-18    
          1000     30      1e-20    
          1024     50      1e-22
         ])
#     sweep = Sweeps([
#          "maxdim" "mindim" "cutoff" 
#           1024      10       1e-16    
#           1024      10      1e-16    
#           1024      20      1e-18    
#           1000     30      1e-20    
#           1024     50      1e-22
#          ])
    eₗ₀,ψₗ = dmrg(Hₗ,ψₗ₀,sweep,outputlevel=1)
    eₛ₀,ψₛ = dmrg(Hₛ,ψₛ₀,sweep,outputlevel=1)
    # error computation
    nrm = norm(inner(Hₛ,ψₛ,Hₛ,ψₛ) - inner(ψₛ',Hₛ,ψₛ)^2)
    #  Overlap computation
    v1 = ITensor(sites[l+1]) 
    for j in 1:size(v1)[1]
        v1[j] = 1/sqrt(2)
    end
    v2 = ITensor(sites[L])
    for j in 1:size(v2)[1]
        v2[j] = 1/sqrt(2)
    end
    v = ITensor(1.0)
    for j in 1:l
        v = v*dag(ψₛ[j])*ψₗ[j]
    end
    v = v*v1*ψₗ[l+1]
    v = v*v2*ψₗ[L]
    return inner(v,v), norm(v), nrm 
end

GrossNeveu (generic function with 1 method)

In [3]:
result_inner, result_norm, result_error = nothing, nothing, nothing

(nothing, nothing, nothing)

In [4]:
# L::Integer, N::Integer,G²::Float64,m::Float64,sw::Integer
result_inner = Vector{ComplexF64}();
result_norm = Vector{ComplexF64}();
result_error = Vector{ComplexF64}();

In [ ]:
# GrossNeveu(L,N,m,G²)
for j in 4:100
    println(j)
    nnr , nrm, err = GrossNeveu(j,1,0.8,1.5); # change BD --> 100
    push!(result_inner, nnr);
    push!(result_norm, nrm);
    push!(result_error, err);
end 

4
After sweep 1 energy=-1.4523216046226433  maxlinkdim=4 maxerr=0.00E+00 time=40.872
After sweep 2 energy=-1.4664345817765538  maxlinkdim=4 maxerr=0.00E+00 time=0.009
After sweep 3 energy=-1.4664796568291396  maxlinkdim=4 maxerr=0.00E+00 time=0.009
After sweep 4 energy=-1.4664798034332298  maxlinkdim=4 maxerr=0.00E+00 time=0.009
After sweep 5 energy=-1.4664798039106766  maxlinkdim=4 maxerr=0.00E+00 time=0.009
After sweep 1 energy=-0.7101363049718934  maxlinkdim=2 maxerr=0.00E+00 time=5.921
After sweep 2 energy=-0.7115654469930859  maxlinkdim=2 maxerr=0.00E+00 time=0.002
After sweep 3 energy=-0.711565690563585  maxlinkdim=2 maxerr=0.00E+00 time=0.002
After sweep 4 energy=-0.7115656906050289  maxlinkdim=2 maxerr=0.00E+00 time=0.002
After sweep 5 energy=-0.7115656906050358  maxlinkdim=2 maxerr=0.00E+00 time=0.002
5
After sweep 1 energy=-2.3980256808587135  maxlinkdim=4 maxerr=0.00E+00 time=0.011
After sweep 2 energy=-2.4033629524483646  maxlinkdim=4 maxerr=0.00E+00 time=0.013
After sweep 

14
After sweep 1 energy=-5.472208097849852  maxlinkdim=100 maxerr=1.23E-07 time=0.750
After sweep 2 energy=-5.4826176848568755  maxlinkdim=124 maxerr=7.27E-17 time=0.829
After sweep 3 energy=-5.483554274374064  maxlinkdim=126 maxerr=9.59E-19 time=0.770
After sweep 4 energy=-5.483664461491515  maxlinkdim=128 maxerr=0.00E+00 time=0.737
After sweep 5 energy=-5.483675708488402  maxlinkdim=127 maxerr=6.77E-23 time=0.755
After sweep 1 energy=-4.650378536313916  maxlinkdim=64 maxerr=0.00E+00 time=0.312
After sweep 2 energy=-4.674121987097713  maxlinkdim=64 maxerr=0.00E+00 time=0.241
After sweep 3 energy=-4.677586247317766  maxlinkdim=64 maxerr=0.00E+00 time=0.248
After sweep 4 energy=-4.67796692268625  maxlinkdim=64 maxerr=0.00E+00 time=0.227
After sweep 5 energy=-4.677994046107884  maxlinkdim=64 maxerr=0.00E+00 time=0.274
15
After sweep 1 energy=-6.401574043844609  maxlinkdim=100 maxerr=1.24E-06 time=1.169
After sweep 2 energy=-6.426877244008453  maxlinkdim=128 maxerr=0.00E+00 time=1.302
Aft

After sweep 5 energy=-8.848995083572838  maxlinkdim=1022 maxerr=9.73E-23 time=96.876
24
After sweep 1 energy=-9.495883579162614  maxlinkdim=100 maxerr=4.55E-04 time=6.399
After sweep 2 energy=-9.511577631874307  maxlinkdim=200 maxerr=7.81E-12 time=15.656
After sweep 3 energy=-9.512076730671959  maxlinkdim=500 maxerr=3.14E-16 time=57.316
After sweep 4 energy=-9.512127123149865  maxlinkdim=1000 maxerr=2.24E-18 time=223.252
After sweep 5 energy=-9.51213804805362  maxlinkdim=1024 maxerr=1.87E-18 time=293.835
After sweep 1 energy=-8.636519265808232  maxlinkdim=100 maxerr=2.26E-04 time=4.626
After sweep 2 energy=-8.695025903600179  maxlinkdim=200 maxerr=1.22E-10 time=9.861
After sweep 3 energy=-8.703483567158058  maxlinkdim=500 maxerr=2.45E-14 time=39.064
After sweep 4 energy=-8.705610853322117  maxlinkdim=1000 maxerr=1.09E-16 time=124.184
After sweep 5 energy=-8.706209517622236  maxlinkdim=1024 maxerr=5.10E-17 time=156.078
25
After sweep 1 energy=-10.442484270561097  maxlinkdim=100 maxerr=4

After sweep 1 energy=-12.839373186194374  maxlinkdim=100 maxerr=6.13E-04 time=9.537
After sweep 2 energy=-12.87337789215981  maxlinkdim=200 maxerr=8.92E-11 time=26.249
After sweep 3 energy=-12.876504027005835  maxlinkdim=500 maxerr=1.17E-13 time=101.674
After sweep 4 energy=-12.87717256617555  maxlinkdim=1000 maxerr=1.35E-14 time=524.744
After sweep 5 energy=-12.87737189650376  maxlinkdim=1024 maxerr=9.99E-15 time=719.483
34
After sweep 1 energy=-13.506030417240032  maxlinkdim=100 maxerr=7.95E-04 time=11.200
After sweep 2 energy=-13.536383324866694  maxlinkdim=200 maxerr=5.61E-11 time=29.867
After sweep 3 energy=-13.539624787229071  maxlinkdim=500 maxerr=1.74E-13 time=124.463
After sweep 4 energy=-13.54031096247981  maxlinkdim=1000 maxerr=2.21E-14 time=574.723
After sweep 5 energy=-13.540512513608752  maxlinkdim=1024 maxerr=2.00E-14 time=925.022
After sweep 1 energy=-12.670249729166045  maxlinkdim=100 maxerr=6.52E-04 time=9.054
After sweep 2 energy=-12.725163478450174  maxlinkdim=200 m

After sweep 2 energy=-17.69924162133545  maxlinkdim=200 maxerr=1.35E-11 time=41.347
After sweep 3 energy=-17.7076828129  maxlinkdim=500 maxerr=1.14E-13 time=185.851
After sweep 4 energy=-17.71031157183891  maxlinkdim=1000 maxerr=8.11E-14 time=845.587
After sweep 5 energy=-17.71118103371336  maxlinkdim=1024 maxerr=8.29E-14 time=1514.597
After sweep 1 energy=-16.859711550899764  maxlinkdim=100 maxerr=8.86E-04 time=14.399
After sweep 2 energy=-16.899324274655648  maxlinkdim=200 maxerr=2.82E-11 time=39.418
After sweep 3 energy=-16.904195278745508  maxlinkdim=500 maxerr=2.24E-13 time=170.815
After sweep 4 energy=-16.905433199689345  maxlinkdim=1000 maxerr=1.14E-13 time=908.051
After sweep 5 energy=-16.90578394025301  maxlinkdim=1024 maxerr=8.60E-14 time=1382.092
44
After sweep 1 energy=-17.536544577306685  maxlinkdim=100 maxerr=9.09E-04 time=16.520
After sweep 2 energy=-17.56641098170196  maxlinkdim=200 maxerr=4.29E-12 time=45.217
After sweep 3 energy=-17.568385128541998  maxlinkdim=500 max

After sweep 3 energy=-19.981907275608158  maxlinkdim=500 maxerr=2.27E-14 time=234.893
After sweep 4 energy=-19.98468555576869  maxlinkdim=1000 maxerr=3.16E-14 time=1270.671
After sweep 5 energy=-19.98561965692634  maxlinkdim=1024 maxerr=6.79E-14 time=1773.761
53
After sweep 1 energy=-21.645827729568058  maxlinkdim=100 maxerr=9.79E-04 time=21.561
After sweep 2 energy=-21.71048573617752  maxlinkdim=200 maxerr=2.75E-12 time=56.053
After sweep 3 energy=-21.72869671258098  maxlinkdim=500 maxerr=4.72E-14 time=257.510
After sweep 4 energy=-21.735887254095655  maxlinkdim=1000 maxerr=6.25E-14 time=1431.566
After sweep 5 energy=-21.738573898269127  maxlinkdim=1024 maxerr=1.28E-13 time=2123.638
After sweep 1 energy=-20.842332109652954  maxlinkdim=100 maxerr=8.32E-04 time=19.946
After sweep 2 energy=-20.91506245366845  maxlinkdim=200 maxerr=5.66E-12 time=55.835
After sweep 3 energy=-20.928480501452366  maxlinkdim=500 maxerr=7.46E-14 time=247.039
After sweep 4 energy=-20.93248628413791  maxlinkdim=

After sweep 3 energy=-24.8168075890469  maxlinkdim=500 maxerr=1.09E-14 time=329.057
After sweep 4 energy=-24.819059005948535  maxlinkdim=1000 maxerr=2.99E-14 time=1792.332
After sweep 5 energy=-24.819846102625355  maxlinkdim=1024 maxerr=3.86E-14 time=2749.021
After sweep 1 energy=-23.91530685610506  maxlinkdim=100 maxerr=9.30E-04 time=25.009
After sweep 2 energy=-23.97549742372663  maxlinkdim=200 maxerr=2.43E-12 time=73.573
After sweep 3 energy=-23.99814706991996  maxlinkdim=500 maxerr=2.39E-14 time=316.495
After sweep 4 energy=-24.008664632343244  maxlinkdim=1000 maxerr=6.07E-14 time=1712.946
After sweep 5 energy=-24.012628007962856  maxlinkdim=1024 maxerr=1.91E-13 time=2273.265
63
After sweep 1 energy=-25.723597329780695  maxlinkdim=100 maxerr=9.08E-04 time=27.189
After sweep 2 energy=-25.7596278595049  maxlinkdim=200 maxerr=5.91E-13 time=71.570
After sweep 3 energy=-25.765671519568414  maxlinkdim=500 maxerr=1.79E-14 time=330.145
After sweep 4 energy=-25.767487397013028  maxlinkdim=1

After sweep 4 energy=-28.184191578409944  maxlinkdim=1000 maxerr=4.52E-15 time=2211.606
After sweep 5 energy=-28.18513460378645  maxlinkdim=1024 maxerr=1.32E-14 time=3392.201
72
After sweep 1 energy=-28.80629419725359  maxlinkdim=100 maxerr=7.85E-04 time=32.497
After sweep 2 energy=-28.841892285840373  maxlinkdim=200 maxerr=2.30E-13 time=89.122
After sweep 3 energy=-28.846556341419287  maxlinkdim=500 maxerr=1.52E-15 time=385.175
After sweep 4 energy=-28.84803733787519  maxlinkdim=1000 maxerr=1.94E-15 time=2244.239
After sweep 5 energy=-28.84851389699795  maxlinkdim=1024 maxerr=6.10E-15 time=3037.328
After sweep 1 energy=-27.934017387469037  maxlinkdim=100 maxerr=1.03E-03 time=28.695
After sweep 2 energy=-28.012076393495608  maxlinkdim=200 maxerr=4.11E-12 time=84.519
After sweep 3 energy=-28.032018982493717  maxlinkdim=500 maxerr=4.38E-14 time=379.998
After sweep 4 energy=-28.039571240191076  maxlinkdim=1000 maxerr=3.52E-14 time=2163.765
After sweep 5 energy=-28.04202105086884  maxlinkd

In [73]:
result_inner2, result_norm2, result_error2 = nothing, nothing, nothing

(nothing, nothing, nothing)

In [74]:
site_number = [x for x in 1:length(result_inner)];
result_inner2 = real(result_inner);
result_norm2 = real(result_norm);
result_error2 = real(result_error);

In [75]:
error_bar = nothing

In [76]:
error_bar = [sqrt(x) for x in result_error2[1:end]];

In [86]:
scatter(site_number, 
    result_norm2.± error_bar, 
    xlim=(0,35),xticks=((0:5:35)), 
    ylim=(0.34,0.49), 
    yticks=((0.34:0.01:0.49)),   
    markersize=2,
    xlabel="Site Number",
    ylabel="Inner Product",
    label="m₀=0.5,G²=1.5",
    title="Overlap")
# savefig("m05_2.svg")

"D:\\Amin\\GN\\G^2\\1.5\\m05_2.svg"

In [87]:
# writedlm("m05_inner.txt",result_inner2)
# writedlm("m05_norm.txt",result_norm2)
# writedlm("m05_error.txt",result_error2)